In [1]:
#import everything I need
import requests
import pickle
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
class player: #class for players
    def __init__(self):
        self
class game: #each object - a game between two players
    def __init__(self):
        self
#functions for files    
def writefile(file, data):
    with open(file+'.pkl', 'wb') as output:
        #save = data
        pickle.dump(data, output, pickle.HIGHEST_PROTOCOL)
        
def readfile(file):
    with open(file+'.pkl', 'rb') as input:
        return(pickle.load(input))  
    
#returns dataframe with the general data on each player in the ttligapro database    
def playerspkltopd(file):
    players = readfile(file)
    pzerodict = players[0].__dict__
    columns = []
    for value, key in zip(pzerodict.values(), pzerodict.keys()):
        if str(type(value))=="<class 'str'>":
            columns.append(key)
            try:
                c.update({key: []})
            except: #occurs when the dictionary is not created yet
                c = {key: []}
            for pl in players:
                c[key].append(pl.__dict__[key])
    df = pd.DataFrame(data=c)
    return(df)  
def playspkltopd(file):
    players = readfile(file)
    for pl in players: #these five lines delete players, who haven't played a single match
        try:
            a = pl.against.values()
        except:
            players.remove(pl)
    i=0
    c1 = {}
    for pl in players:
        for p, k in zip(pl.against.values(), pl.against.keys()):
            for m in range(len(p)):
                o = [pl.id, k]
                o += p[m]
                c1.update({str(i): o})
                i+=1    
    df = pd.DataFrame(data=c1)
    df.rename(index={0:'id1',1:'id2',2:'date',3:'tournament_name',\
                     4:'current_rating1',5:'current_rating2',6:'score_sets',7:'score_points',\
                    8:'rating_delta_for1'}, inplace=True)
    df = df.T
    dfplays = df
    dfplayers1 = dfplayers.copy()
    dfplayers2 = dfplayers.copy()
    dfplayers1.columns = [i+'1' for i in dfplayers.columns]   
    dfplayers2.columns = [i+'2' for i in dfplayers.columns]   
    dfplays = pd.merge(dfplays, dfplayers2, on=['id2'])
    dfplays = pd.merge(dfplays, dfplayers1, on=['id1'])
    dfplays['won_sets'] = [a[0] for a in dfplays['score_sets']]
    dfplays['lost_sets'] = [a[4] for a in dfplays['score_sets']]
    return(dfplays)

In [3]:
dfplayers = playerspkltopd('plfull')
dfplays = playspkltopd('plfull')
writefile('players_pandas', dfplayers)
writefile('plays_pandas', dfplays)